### import bibliotek

In [ ]:
import os
import json
import requests 
from bs4 import BeautifulSoup

### Wysłanie żądania dostępu do strony internretowej z opiniami o produkcie 

In [ ]:


headers = {
    "Cookie":"sv3=1.0_6bd134ff-151f-11f0-9e99-7c1c3b6fec11; __RequestVerificationToken=wwLDXsg3Uuah3cEzW1fqvJTcJ90KeXJL3Ab6aMhtNUoaU-9fp8nCTPGVkLefnluqOz6m5FpwYLlLsAfChZ8KIcrkJifnQqCaWA0FvuH8vTQ1; userCeneo=ID=d7679f25-accf-47e3-b9ff-195986a8a19f; __utmf=a74e177af2dad576aa770a2ad74bd8eb_k2wCRI6tAVSgxOOwMsWh%2Bvo35Yf981ST; appType=%7B%22Value%22%3A1%7D; cProdCompare_v2=; browserBlStatus=0; __eoi=ID=f486909af682d15b:T=1744188518:RT=1744188518:S=AA-AfjY0dnqIkDBA-pXcq6f2BZGt; _gcl_au=1.1.540495262.1744188522; consentcookie=eyJBZ3JlZUFsbCI6dHJ1ZSwiQ29uc2VudHMiOlsxLDMsNCwyXSwiVENTdHJpbmciOiJDUVBtVFVBUVBtVFVBR3lBQkJQTEJrRXNBUF9nQUFBQUFCNVlJTnBEN0JiQkxVRkF3RmhqWUtzUU1JRVRVTUNBQW9RQUFBYUJBQ0FCUUFLUUlBUUNra0FRQkFTZ0JBQUNBQUFBSUNSQklRQU1BQUFBQ0VBQVFBQUFJQUFFQUFDUUJRQUlBQUFBZ0FBUUFBQVlBQUFpQUlBQUFBQUlnQUlBRUFBQW1RaEFBQUlBRUVBQWhBQUVJQUFBQUFBQUFBQUFBZ0FBQUFBQ0FBSUFBQUFBQUNBQUFJSU5nQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUJZS0FEQUFFRUd3a0FHQUFJSU5ob0FNQUFRUWJFUUFZQUFnZzJLZ0F3QUJCQnNaQUJnQUNDRFk2QURBQUVFR3lFQUdBQUlJTmtvQU1BQVFRYktRQVlBQWdnMldnQXdBQkJCc0EiLCJWZXJzaW9uIjoidjMifQ==; nps3=SessionStartTime=1744188573,SurveyId=64",
    "Host":"www.ceneo.pl",
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36 OPR/117.0.0.0"
}

In [ ]:
product_id = '84514582'
url = f'https://www.ceneo.pl/{product_id}#tab=reviews_scroll'
all_opinions = []
response = requests.get(url, headers=headers)

### Jeżeli operacja zakończy się powozeniem, weodrębnienie w kodu strony opini o produkcie 


In [ ]:
if response.status_code == 200:
    page_dom = BeautifulSoup(response.text, 'html.parser')
    opinions = page_dom.select('div.js_product-review:not(.user-post--highlight)')

### Dla każdej opinii wyodrębnienie z kodu HTML poszczególnych składowych i przysylanie ich do elementów zlożonej struktury danych


In [ ]:
for opinion in opinions:
    try:
        single_opinion = {
        "author" : opinion.select_one("span.user-post__author-name").text.strip(),
        "recommendation" : opinion.select_one("span.user-post__author-recomendation > em").text.strip(),
        "stars" : opinion.select_one("span.user-post__score-count").text.strip(),
        "content" : opinion.select_one("div.user-post__text").text.strip(),
        "pros" : [p.text.strip() for p in opinion.select("div.review-feature__item--positive")],
        "cons" : [c.text.strip() for c in opinion.select("div.review-feature__item--negative")],
        "useful" : opinion.select_one("button.vote-yes > span").text.strip(),
        "unuseful" : opinion.select_one("button.vote-no > span").text.strip(),
        "post_date" : opinion.select_one("span.user-post__published > time:nth-of-type(1)")['datetime'].strip(),
        "purchase_date" : opinion.select_one("span.user-post__published > time:nth-of-type(2)")['datetime'].strip(),
        }
        all_opinions.append(single_opinion)
        print(single_opinion)
    except (AttributeError, TypeError):
        pass
    

In [ ]:
print(type(single_opinion['author']))

### Jeśli istneje kolejna strona z opiniami, przejście do niej i powtórzen dla niej kroków 1-4


In [ ]:
product_id = "84514582"
next_page = (f"https://www.ceneo.pl/{product_id}#tab=reviews")
all_opinions = []
while next_page:
    response = requests.get(next_page,headers=headers)
    if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, "html.parser")
        opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")
        for opinion in opinions:
            try:
                single_opinion = {
                    "author": opinion.select_one("span.user-post__author-name").text.strip(),
                    "recommendation": opinion.select_one("span.user-post__author-recomendation > em").text.strip(),
                    "stars": opinion.select_one("span.user-post__score-count").text.strip(),
                    "content": opinion.select_one("div.user-post__text").text.strip(),
                    "pros": [p.text.strip() for p in opinion.select("div.review-feature__item--positive")],
                    "cons": [c.text.strip() for c in opinion.select("div.review-feature__item--negative")],
                    "useful": opinion.select_one("button.vote-yes > span").text.strip(),
                    "useless": opinion.select_one("button.vote-no > span").text.strip(),
                    "post_date": opinion.select_one("span.user-post__published > time:nth-child(1)")['datetime'].strip(),
                    "purchase_date": opinion.select_one("span.user-post__published > time:nth-child(2)")['datetime'].strip(),
                }
                all_opinions.append(single_opinion)
            except (AttributeError, TypeError):
                pass
        next_page_element = page_dom.select_one("a.pagination__next")
        try:
            next_page = "https://www.ceneo.pl" + next_page_element["href"]
            print(next_page)
        except TypeError:
            next_page = None
            print("Brak kolejnej strony.")
    else:
        print(f"{response.status_code}")

### Zapisanie wyników do bazy danych 


In [ ]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")

In [ ]:
with open(f"./opinions/{product_id}.json", "w", encoding="UTF-8") as jf:
    json.dump(all_opinions, jf, indent=4, ensure_ascii=False)